In [ ]:
#Import the libraries 
#import pandas_datareader as web
import numpy as np
import pandas as pd
import statsmodels.api as sm
#!pip install yahoofinance
#import yahoofinance as yf



In [ ]:
#Get the stock quote
##df = web.DataReader('TSLA', data_source='yahoo', start='2010-01-01',end='2019-12-31')

In [ ]:
from google.colab import files
dataset = files.upload()


Saving TSLA.csv to TSLA (1).csv


In [ ]:
data = pd.read_csv('TSLA.csv')
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2010-07-01,5.000000,5.184000,4.054000,4.392000,4.392000,41094000
1,2010-07-02,4.600000,4.620000,3.742000,3.840000,3.840000,25699000
2,2010-07-06,4.000000,4.000000,3.166000,3.222000,3.222000,34334500
3,2010-07-07,3.280000,3.326000,2.996000,3.160000,3.160000,34608500
4,2010-07-08,3.228000,3.504000,3.114000,3.492000,3.492000,38557000
...,...,...,...,...,...,...,...
2387,2019-12-24,83.671997,85.094002,82.538002,85.050003,85.050003,40273500
2388,2019-12-26,85.582001,86.695999,85.269997,86.188004,86.188004,53169500
2389,2019-12-27,87.000000,87.061996,85.222000,86.075996,86.075996,49728500
2390,2019-12-30,85.758003,85.800003,81.851997,82.940002,82.940002,62932000


Сalculating the %change, mulitply by 100 to get value in %


In [ ]:
df=data['Adj Close'].pct_change()*100 
df

0             NaN
1      -12.568306
2      -16.093750
3       -1.924271
4       10.506329
          ...    
2387     1.438387
2388     1.338038
2389    -0.129958
2390    -3.643285
2391     0.875329
Name: Adj Close, Length: 2392, dtype: float64

Сonvert from series to data frame and rename the column


In [ ]:
df = pd.DataFrame(df) 
df
df.rename(columns={'Adj Close':'Today'})


,Today
0,NaN
1,-12.568306
2,-16.093750
3,-1.924271
4,10.506329
...,...
2387,1.438387
2388,1.338038
2389,-0.129958
2390,-3.643285


In [ ]:
extracted_col = data["Date"]
extracted_col=pd.DataFrame(extracted_col)
extracted_col


,Date
0,2010-07-01
1,2010-07-02
2,2010-07-06
3,2010-07-07
4,2010-07-08
...,...
2387,2019-12-24
2388,2019-12-26
2389,2019-12-27
2390,2019-12-30


There are no common values, the join operation fails because the values don't overlap it requires you to supply a suffix for the left and right hand side
so we can use merge function since it does not have the restriction

In [ ]:
#df.merge(df, extracted_col, how='outer',on='Date')

df = pd.concat([df, extracted_col], axis=1, join='inner')
df

,Adj Close,Date
0,NaN,2010-07-01
1,-12.568306,2010-07-02
2,-16.093750,2010-07-06
3,-1.924271,2010-07-07
4,10.506329,2010-07-08
...,...,...
2387,1.438387,2019-12-24
2388,1.338038,2019-12-26
2389,-0.129958,2019-12-27
2390,-3.643285,2019-12-30


In [ ]:
#reassign the dataframe with a list of the columns
cols=df.columns.tolist()
cols

['Adj Close', 'Date']

In [ ]:
#Rearrange cols in any way you want


cols = cols[-1:] + cols[:1]
cols
df=df[cols]
df
#df.rename(columns={'Adj Close':'Today'})

,Date,Adj Close
0,2010-07-01,NaN
1,2010-07-02,-12.568306
2,2010-07-06,-16.093750
3,2010-07-07,-1.924271
4,2010-07-08,10.506329
...,...,...
2387,2019-12-24,1.438387
2388,2019-12-26,1.338038
2389,2019-12-27,-0.129958
2390,2019-12-30,-3.643285



Lagged values are used in Dynamic Regression modeling. They are also used when it is assumed that the forecast of the next period depends on past values of the same series.


In statistics a distributed lag model is a model for time series data in which a regression equation is used to predict current values of a dependent variable based on both the current values of an predictor variable and the lagged (past period) values of this predictor variable.

The prior day return of day 2 is value of day 1, so we can shift value by the lag. So lag1 value for day 2 is the value of %change of day 1 

In [ ]:
for i in range(1,6):
  df['Lag '+ str(i)]= df['Adj Close'].shift(i)
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,Adj Close,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5
0,2010-07-01,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-07-02,-12.568306,NaN,NaN,NaN,NaN,NaN
2,2010-07-06,-16.093750,-12.568306,NaN,NaN,NaN,NaN
3,2010-07-07,-1.924271,-16.093750,-12.568306,NaN,NaN,NaN
4,2010-07-08,10.506329,-1.924271,-16.093750,-12.568306,NaN,NaN
...,...,...,...,...,...,...,...
2387,2019-12-24,1.438387,3.360544,0.383622,2.769938,3.736246,-0.657937
2388,2019-12-26,1.338038,1.438387,3.360544,0.383622,2.769938,3.736246
2389,2019-12-27,-0.129958,1.338038,1.438387,3.360544,0.383622,2.769938
2390,2019-12-30,-3.643285,-0.129958,1.338038,1.438387,3.360544,0.383622


In [ ]:
df['Volume'] = data.Volume.shift(1).values/1000_000_0
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Adj Close,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
0,2010-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-07-02,-12.568306,NaN,NaN,NaN,NaN,NaN,4.10940
2,2010-07-06,-16.093750,-12.568306,NaN,NaN,NaN,NaN,2.56990
3,2010-07-07,-1.924271,-16.093750,-12.568306,NaN,NaN,NaN,3.43345
4,2010-07-08,10.506329,-1.924271,-16.093750,-12.568306,NaN,NaN,3.46085
...,...,...,...,...,...,...,...,...
2387,2019-12-24,1.438387,3.360544,0.383622,2.769938,3.736246,-0.657937,6.65980
2388,2019-12-26,1.338038,1.438387,3.360544,0.383622,2.769938,3.736246,4.02735
2389,2019-12-27,-0.129958,1.338038,1.438387,3.360544,0.383622,2.769938,5.31695
2390,2019-12-30,-3.643285,-0.129958,1.338038,1.438387,3.360544,0.383622,4.97285


In [ ]:
df=df.dropna()
df

,Date,Adj Close,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume
6,2010-07-12,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,-12.568306,2.02530
7,2010-07-13,6.392962,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,1.10125
8,2010-07-14,9.371555,6.392962,-2.011494,-0.343643,10.506329,-1.924271,1.34005
9,2010-07-15,0.252016,9.371555,6.392962,-2.011494,-0.343643,10.506329,2.09760
10,2010-07-16,3.770739,0.252016,9.371555,6.392962,-2.011494,-0.343643,1.86990
...,...,...,...,...,...,...,...,...
2387,2019-12-24,1.438387,3.360544,0.383622,2.769938,3.736246,-0.657937,6.65980
2388,2019-12-26,1.338038,1.438387,3.360544,0.383622,2.769938,3.736246,4.02735
2389,2019-12-27,-0.129958,1.338038,1.438387,3.360544,0.383622,2.769938,5.31695
2390,2019-12-30,-3.643285,-0.129958,1.338038,1.438387,3.360544,0.383622,4.97285


In [ ]:
df['Direction']=[1 if i>0 else 0 for i in df['Adj Close']]
df


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Date,Adj Close,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction
6,2010-07-12,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,-12.568306,2.02530,0
7,2010-07-13,6.392962,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,1.10125,1
8,2010-07-14,9.371555,6.392962,-2.011494,-0.343643,10.506329,-1.924271,1.34005,1
9,2010-07-15,0.252016,9.371555,6.392962,-2.011494,-0.343643,10.506329,2.09760,1
10,2010-07-16,3.770739,0.252016,9.371555,6.392962,-2.011494,-0.343643,1.86990,1
...,...,...,...,...,...,...,...,...,...
2387,2019-12-24,1.438387,3.360544,0.383622,2.769938,3.736246,-0.657937,6.65980,1
2388,2019-12-26,1.338038,1.438387,3.360544,0.383622,2.769938,3.736246,4.02735,1
2389,2019-12-27,-0.129958,1.338038,1.438387,3.360544,0.383622,2.769938,5.31695,0
2390,2019-12-30,-3.643285,-0.129958,1.338038,1.438387,3.360544,0.383622,4.97285,0


In [ ]:
# we need add constant for the regression model so that it contains intercept
df=sm.add_constant(df)
df

#this is a column that just contains ones.
#Now we have adjusted data frame to perform a regression


,const,Date,Adj Close,Lag 1,Lag 2,Lag 3,Lag 4,Lag 5,Volume,Direction
6,1.0,2010-07-12,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,-12.568306,2.02530,0
7,1.0,2010-07-13,6.392962,-2.011494,-0.343643,10.506329,-1.924271,-16.093750,1.10125,1
8,1.0,2010-07-14,9.371555,6.392962,-2.011494,-0.343643,10.506329,-1.924271,1.34005,1
9,1.0,2010-07-15,0.252016,9.371555,6.392962,-2.011494,-0.343643,10.506329,2.09760,1
10,1.0,2010-07-16,3.770739,0.252016,9.371555,6.392962,-2.011494,-0.343643,1.86990,1
...,...,...,...,...,...,...,...,...,...,...
2387,1.0,2019-12-24,1.438387,3.360544,0.383622,2.769938,3.736246,-0.657937,6.65980,1
2388,1.0,2019-12-26,1.338038,1.438387,3.360544,0.383622,2.769938,3.736246,4.02735,1
2389,1.0,2019-12-27,-0.129958,1.338038,1.438387,3.360544,0.383622,2.769938,5.31695,0
2390,1.0,2019-12-30,-3.643285,-0.129958,1.338038,1.438387,3.360544,0.383622,4.97285,0


In [ ]:
#We want to use the lags and the volume to predict the direction,
#So lags and volume are independent variables and direction is dependent

In [ ]:
#Defining independent variables
X=df[['const','Lag 1','Lag 2','Lag 3','Lag 4','Lag 5','Volume']]
#Defining dependent variable
y=df.Direction

In [ ]:
#define the model
model=sm.Logit(y,X)

In [ ]:
# Fitting the model
result=model.fit()

Optimization terminated successfully.
         Current function value: 0.692109
         Iterations 3


In [ ]:
#Look at the summary to get coeff of independt variables 
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                 2386
Model:                          Logit   Df Residuals:                     2379
Method:                           MLE   Df Model:                            6
Date:                Tue, 09 Nov 2021   Pseudo R-squ.:                0.001155
Time:                        20:59:44   Log-Likelihood:                -1651.4
converged:                       True   LL-Null:                       -1653.3
Covariance Type:            nonrobust   LLR p-value:                    0.7009
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0394      0.063      0.630      0.529      -0.083       0.162
Lag 1         -0.0169      0.013     -1.319      0.187      -0.042       0.008
Lag 2         -0.0038      0.013     -0.301      0.764      -0.029       0.021
Lag 3         -0.0178      0.013     -1.386      0.166      -0.043       0.007
Lag 4         -0.0030      0.013     -0.233      0.816      -0.028       0.022
Lag 5         -0.0043      0.013     -0.341      0.733      -0.029       0.021
Volume         0.0047      0.018      0.269      0.788      -0.030       0.039
==============================================================================
"""

In [ ]:
prediction = result.predict(X)
prediction
#In the logistic regression if the value is above 0.5 than the direction is up

6       0.537539
7       0.492153
8       0.482166
9       0.464458
10      0.475490
          ...   
2387    0.488771
2388    0.497494
2389    0.490884
2390    0.505682
2391    0.522137
Length: 2386, dtype: float64

In [ ]:
#To evaluate whether the prediction is good or bad we can use confusion matrix
#It compares actual values with predictor values

def confusion_matrix(actual,predicted):
  predicted_trans =['Up' if i>0.5 else "Down" for i in predicted]
  actuals = ['Up' if i>0 else "Down" for i in actual]
  confusion_matrix=pd.crosstab(pd.Series(actuals),
                               pd.Series(predicted_trans),
                               rownames=['Actual'],
                               colnames=['Predicted'])
  return confusion_matrix

In [ ]:
 #diagonal are the right predictions 
 confusion_matrix(y,prediction)

Predicted,Down,Up
Actual,,
Down,312,855
Up,302,917


In [ ]:
len(df)
(315+913)/2385

0.5148846960167714

In [ ]:
#The model's accuracy is 51.48%. Which is not great. But! it is better than random guessing


In [ ]:
# Now we can train and test the model on the same set of observations to avoid an "overly optimistic model".
# We can split data in train and test data set
# train data set contains dates before 2005 and test contains in 2005. So we use data before to check what the mode predicts to compare what the model predicts.
x_train=df[df.Date.dt.year<2019][['const','Lag 1','Lag 2','Lag 3','Lag 4','Lag 5','Volume']]
y_train=df[df.Date.dt.year<2019]['Direction']
x_test=df[df.Date.dt.year==2019][['const','Lag 1','Lag 2','Lag 3','Lag 4','Lag 5','Volume']]
y_test=df[df.Date.dt.year==2019]['Direction']

AttributeError: ignored

In [ ]:
model=sm.Logit(y_train, x_train)

In [ ]:
result=model.fit()

In [ ]:
#Now we provide the test data
prediction=result.predict(x_test)


In [ ]:
#created confusion matrix with test value and predicted value
confusion_matrix(y_test,prediction)


In [ ]:
len(x_test)


In [ ]:
#WE can cut off variable to check if it affects the accuracy of the model 
